# Massachusetts Colleges Capstone Project - Raul Gonzalez
# Data
# April 7th, 2020

The data that will be used for this project will be a List of colleges and universities in Massachusetts published in Wikipedia in the following link: https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Massachusetts. 

I will use the first table which comprises all the active colleges and universities in Massachusetts. Lets load, visualize and examine the data.

## First I will extract all the data from Wikipedia using pandas, in specific the tables

### Lets import the required libraries

In [16]:
# extract tables from wikipedia
from pandas.io.html import read_html

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import re

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

#here I extract the tables from the wikipedia site
page = 'https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Massachusetts'

wikitables = read_html(page,  attrs={"class":"wikitable sortable"})

print ("Extracted {num} wikitables".format(num=len(wikitables)))

Libraries imported.
Extracted 3 wikitables


I will only use the first table for this project

In [17]:
#Lets visualize the table without cleaning and preprocessing
df=wikitables[0]
print(df.shape)
df.head(10)

(104, 7)


,School,Location[note 1],Control[1],Type[1],Enrollment[16],Founded,Accreditation[16]
0,American International College,Springfield,Private not-for-profit,Master's university,"2,177[17]",1885[17],"AOTA, APTA, CCNE, NEASC[17]"
1,Amherst College,Amherst,Private not-for-profit,Baccalaureate college,"1,817[18]",1821[18],NEASC[18]
2,Anna Maria College,Paxton,Private not-for-profit,Master's university,"1,455[19]",1946[19],"NASM, NEASC, NLNAC[19]"
3,Assumption College,Worcester,Private not-for-profit,Master's university,"2,813[20]",1904[20],NEASC[20]
4,Babson College,Wellesley,Private not-for-profit,Special-focus institution,"3,250[21]",1919[21],NEASC[21]
5,Bard College at Simon's Rock,Great Barrington,Private not-for-profit,Baccalaureate/associate's college,354[22],1964[22],NEASC[22]
6,Bay Path University,Longmeadow,Private not-for-profit,Baccalaureate college,"2,370[23]",1897[23],"AOTA, NEASC[23]"
7,Bay State College,Boston,For-profit,Associate's college,"1,721[24]",1946[24],"ABHES, APTA, NEASC, NLNAC[24]"
8,Becker College,Worcester,Private not-for-profit,Baccalaureate college,"1,826[25]",1784[25],"APTA, NEASC, NLNAC[25]"
9,Benjamin Franklin Institute of Technology,Boston,Private not-for-profit,Special-focus institution,475[26],1908[26],NEASC[26]


### Removing unwanted data

In [18]:
data=df.drop(['Accreditation[16]'], axis=1)
data.rename(columns={"Location[note 1]": "Location", "Control[1]": "Control", "Type[1]": "Type", "Enrollment[16]": "Enrollment"}, inplace=True)
for i in range (data['Enrollment'].shape[0]):
    data['Enrollment'][i] = re.sub("[\(\[].*?[\)\]]", "", data['Enrollment'][i])
for i in range (data['Founded'].shape[0]):
    data['Founded'][i] = re.sub("[\(\[].*?[\)\]]", "", data['Founded'][i])

data['Enrollment'] = data['Enrollment'].str.replace(',','')
data['Founded'] = data['Founded'].str.replace(',','')
data['Founded'][92] = 1975
data = data.astype({"Enrollment":'int64', "Founded":'int64'})

print('Data shape:',data.shape)
print('The data consists of {} colleges in {} different cities'.format(data.shape[0],np.count_nonzero(data['Location'].unique())))
data.head(10)

Data shape: (104, 6)
The data consists of 104 colleges in 53 different cities


,School,Location,Control,Type,Enrollment,Founded
0,American International College,Springfield,Private not-for-profit,Master's university,2177,1885
1,Amherst College,Amherst,Private not-for-profit,Baccalaureate college,1817,1821
2,Anna Maria College,Paxton,Private not-for-profit,Master's university,1455,1946
3,Assumption College,Worcester,Private not-for-profit,Master's university,2813,1904
4,Babson College,Wellesley,Private not-for-profit,Special-focus institution,3250,1919
5,Bard College at Simon's Rock,Great Barrington,Private not-for-profit,Baccalaureate/associate's college,354,1964
6,Bay Path University,Longmeadow,Private not-for-profit,Baccalaureate college,2370,1897
7,Bay State College,Boston,For-profit,Associate's college,1721,1946
8,Becker College,Worcester,Private not-for-profit,Baccalaureate college,1826,1784
9,Benjamin Franklin Institute of Technology,Boston,Private not-for-profit,Special-focus institution,475,1908


## Now I will explore the data

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 6 columns):
School        104 non-null object
Location      104 non-null object
Control       104 non-null object
Type          104 non-null object
Enrollment    104 non-null int64
Founded       104 non-null int64
dtypes: int64(2), object(4)
memory usage: 5.0+ KB


In [20]:
data[['Enrollment', 'Founded']].describe()

,Enrollment,Founded
count,104.000000,104.000000
mean,4780.480769,1913.009615
std,5934.864141,57.049713
min,18.000000,1636.000000
25%,1281.750000,1878.000000
50%,2496.000000,1918.000000
75%,6314.750000,1963.250000
max,32603.000000,1997.000000


In [21]:
print('The oldest college in Massachusetts is {} founded in {}'.format(data['School'][np.argmin(data['Founded'])],np.min(data['Founded'])))
print('\n')
print('The newest college in Massachusetts is {} founded in {}'.format(data['School'][np.argmax(data['Founded'])],np.max(data['Founded'])))
print('\n')
print('The college with the highest enrollment is {} with {} students'.format(data['School'][np.argmax(data['Enrollment'])],np.max(data['Enrollment'])))
print('\n')
print('The college with the lowest enrollment is {} with {} students'.format(data['School'][np.argmin(data['Enrollment'])],np.min(data['Enrollment'])))
print('\n')
print('The average enrollment for a college in Massachusetts is approximately {} students'.format(round(data['Enrollment'].mean())))
print('\n')
print('The total number of students enrolled in Massachusetts is {} students'.format(data['Enrollment'].sum()))

The oldest college in Massachusetts is Harvard University founded in 1636


The newest college in Massachusetts is Olin College founded in 1997


The college with the highest enrollment is Boston University with 32603 students


The college with the lowest enrollment is Conway School of Landscape Design with 18 students


The average enrollment for a college in Massachusetts is approximately 4780 students


The total number of students enrolled in Massachusetts is 497170 students


In [22]:
print('{} college was founded in the 17th century'.format(np.count_nonzero(data['Founded'].between(1601, 1700, inclusive = True))))
print('\n')
print('{} colleges were founded in the 18th century'.format(np.count_nonzero(data['Founded'].between(1701, 1800, inclusive = True))))
print('\n')
print('{} colleges were founded in the 19th century'.format(np.count_nonzero(data['Founded'].between(1801, 1900, inclusive = True))))
print('\n')
print('{} colleges were founded in the 20th century'.format(np.count_nonzero(data['Founded'].between(1901, 2000, inclusive = True))))
print('\n')

1 college was founded in the 17th century


2 colleges were founded in the 18th century


40 colleges were founded in the 19th century


61 colleges were founded in the 20th century




In [23]:
data[['School', 'Location', 'Control', 'Type']].describe()

,School,Location,Control,Type
count,104,104,104,104
unique,104,53,3,6
top,MCPHS University,Boston,Private not-for-profit,Special-focus institution
freq,1,24,72,28


 - We can see that there are 104 distinct colleges in the dataset, in a total of 53 different locations
 - The location with most colleges is Boston with a total of 24 colleges.
 - Out of the 104 colleges, 72 are Private not-for-profit.
 - The most common type of college are Special-focus institutions.

In [24]:
data_location = data.groupby(['Location']).School.nunique()
data_location.sort_values(ascending=False).head()

Location
Boston         24
Worcester       8
Cambridge       6
Springfield     4
Wellesley       3
Name: School, dtype: int64

The cities with most colleges are Boston, Worcester and Cambridge

In [25]:
data_control = data.groupby(['Control']).School.nunique()
data_control.sort_values(ascending=False)

Control
Private not-for-profit    72
Public                    30
For-profit                 2
Name: School, dtype: int64

72 colleges are Private not-for-profit, 30 are Public and 2 are For-profit

In [26]:
data_type = data.groupby(['Type']).School.nunique()
data_type.sort_values(ascending=False)

Type
Special-focus institution            28
Baccalaureate college                21
Master's university                  20
Associate's college                  20
Research university                  14
Baccalaureate/associate's college     1
Name: School, dtype: int64

28 colleges are Special-focus institutions, 22 are Baccalaureate colleges, 20 are Master's universities, 20 are Associate's colleges and 14 are Research universities

## Then, lets prepare the data in a convenient way

In [27]:
data['Number of Colleges'] = data['Location'].map(data['Location'].value_counts())
data.head(10)

,School,Location,Control,Type,Enrollment,Founded,Number of Colleges
0,American International College,Springfield,Private not-for-profit,Master's university,2177,1885,4
1,Amherst College,Amherst,Private not-for-profit,Baccalaureate college,1817,1821,3
2,Anna Maria College,Paxton,Private not-for-profit,Master's university,1455,1946,1
3,Assumption College,Worcester,Private not-for-profit,Master's university,2813,1904,8
4,Babson College,Wellesley,Private not-for-profit,Special-focus institution,3250,1919,3
5,Bard College at Simon's Rock,Great Barrington,Private not-for-profit,Baccalaureate/associate's college,354,1964,1
6,Bay Path University,Longmeadow,Private not-for-profit,Baccalaureate college,2370,1897,1
7,Bay State College,Boston,For-profit,Associate's college,1721,1946,24
8,Becker College,Worcester,Private not-for-profit,Baccalaureate college,1826,1784,8
9,Benjamin Franklin Institute of Technology,Boston,Private not-for-profit,Special-focus institution,475,1908,24


In [28]:
result = data.groupby('Location', sort=True).agg( ','.join)
result = result.reset_index(drop=True)

result.head()

,School,Control,Type
0,"Amherst College,Hampshire College,University o...","Private not-for-profit,Private not-for-profit,...","Baccalaureate college,Baccalaureate college,Re..."
1,Massachusetts School of Law,Private not-for-profit,Special-focus institution
2,"Endicott College,Montserrat College of Art","Private not-for-profit,Private not-for-profit","Master's university,Special-focus institution"
3,"Bay State College,Benjamin Franklin Institute ...","For-profit,Private not-for-profit,Private not-...","Associate's college,Special-focus institution,..."
4,Massachusetts Maritime Academy,Public,Baccalaureate college


## Finally, I'll just keep the cities, the names and number of colleges in each one

In [29]:
data_clean = pd.DataFrame(data[['Location','Number of Colleges']])
data_clean = data_clean.drop_duplicates()
data_clean.sort_values('Location',inplace=True)
data_clean.rename(columns={"Location": "City"}, inplace=True)
data_clean = data_clean.reset_index(drop=True)
data_clean['Colleges'] = result['School']

data_clean.head(10)

,City,Number of Colleges,Colleges
0,Amherst,3,"Amherst College,Hampshire College,University o..."
1,Andover,1,Massachusetts School of Law
2,Beverly,2,"Endicott College,Montserrat College of Art"
3,Boston,24,"Bay State College,Benjamin Franklin Institute ..."
4,Bourne,1,Massachusetts Maritime Academy
5,Bridgewater,1,Bridgewater State University
6,Brighton,1,Saint John's Seminary
7,Brockton,1,Massasoit Community College
8,Brookline,2,"Boston Graduate School of Psychoanalysis,Helle..."
9,Cambridge,6,"Cambridge College,Harvard University,Hult Inte..."


Now that I am done with the data preparation, the next step would be the implementation of the project which will be done for the next week in the second part of the capstone project. For the implementation, I will start by finding the location of each city using geopy and start using the Foursquare API to get the closest venues for each city, as well as the 10 most common venues in each city. Finally, I will use k-means to cluster the different environments offered by the cities of Massachusetts that houses the state´s colleges and universities, seeking to facilitate prospective college students, wanting to study there, the important task of choosing the right college.